<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/bqddf/harness_api_integration_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Colab by default is on an old version of the bigquery API unfortunately, so update to latest version.

In [ ]:
%pip install --upgrade google-cloud-bigquery

## Imports

In [7]:
# This stub (ddfimport) allows the Ddf EE API to be imported.
import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport

# Use this line to import from a branch of the github repository.
# It will git clone the git repository under a google drive path.
# This allows you to modify the source files by opening the file view and
# changing files under /content/gdrive/MyDrive/<branch_name>
# ddfimport.ddf_source_control_pane()

# Alternatively, you can use this line to import from Main.
# If you import from Main, you will not be able to change files, but will not
# need a Google Login for Google Drive.
ddfimport.ddf_import_common()

interactive(children=(Text(value='', description='Email', placeholder='Enter email'), Text(value='', descripti…

Authenticate

In [48]:
import importlib
import eeddf
import bqddf
importlib.reload(eeddf)
importlib.reload(bqddf)

eeddf.initialize_ddf(test_environment = True)

## Create a fake table for testing.

In [53]:
from google.cloud import bigquery
import time

client = bqddf.get_big_query_client()

# Create a minimal schema
schema = [
    bigquery.SchemaField("eval_id", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("baseline_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("experiment_id", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("completion_timestamp", "TIMESTAMP", mode="NULLABLE"),
]

# Modify config values for test.
bqddf._TEST_CONFIG['TABLE'] = f'test_{int(time.time())}'

# table_id should evaluate to project.dataset.test-table
table_id = f"{bqddf._TEST_CONFIG['PROJECT_NAME']}.{bqddf._TEST_CONFIG['DATASET']}.{bqddf._TEST_CONFIG['TABLE']}"
table = bigquery.Table(table_id, schema=schema)

# If it exists, delete it first.
client.delete_table(table_id, not_found_ok=True)

# Create the test table.
table = client.create_table(table)
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

Created table river-sky-386919.harness_test_db.test_1708563688


In [ ]:
## Create fake data.

In [54]:
import json

def test_read_and_write():
  eval_1 = {
    "eval_id" : 'fake_eval_1',
    "baseline_id" : "baseline_2",
    "experiment_id" : "fake_experiment_1"
  }

  eval_2 = {
    "eval_id" : 'fake_eval_2',
  }

  eval_3 = {
    "eval_id" : 'fake_eval_3',
    "baseline_id" : "baseline_3",
    "experiment_id" : "fake_experiment_1",
  }

  errors = bqddf.insert_eval_result(eval_1)
  assert len(errors) == 0, errors

  errors = bqddf.insert_eval_result(eval_2)
  assert len(errors) == 0, errors

  errors = bqddf.insert_eval_result(eval_3)
  assert len(errors) == 0, errors

  result = bqddf.get_eval_result('fake_eval_1')
  assert result.total_rows == 1

  rows = bqddf.get_eval_result('fake_eval_2')
  assert result.total_rows == 1

  rows = bqddf.get_eval_result('fake_eval_3')
  assert result.total_rows == 1


## Run the test.

In [ ]:
test_read_and_write()

## Delete the test table

In [ ]:
client.delete_table(table_id)